In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

def mortgage_amortization(principal, annual_interest_rate, years, extra_payment_month, extra_payment_amount, start_date):
    monthly_interest_rate = annual_interest_rate / 12 / 100
    total_months = years * 12

    # Monthly mortgage payment calculation
    monthly_payment = principal * monthly_interest_rate * (1 + monthly_interest_rate) ** total_months / ((1 + monthly_interest_rate) ** total_months - 1)

    # Initialize variables
    balance = principal
    amortization_schedule = []
    
    for month in range(1, total_months + 1):
        interest_payment = balance * monthly_interest_rate
        principal_payment = monthly_payment - interest_payment

        # Apply extra payment to principal
        if month == extra_payment_month:
            balance -= (principal_payment + extra_payment_amount)
            principal_payment += extra_payment_amount
        else:
            balance -= principal_payment

        amortization_schedule.append({
            'Date': start_date + DateOffset(months=month-1),
            'Month paid': month,
            'Month remaining': total_months-month,
            'Principal Payment': principal_payment,
            'Interest Payment': interest_payment,
            'Balance': balance if balance > 0 else 0
        })

        # break if the balance is 0 or less
        if balance <= 0:
            break

    return pd.DataFrame(amortization_schedule)

In [ ]:
principal = 720000  # initial loan amount
annual_interest_rate = 3.625  # annual interest rate in percent
years = 30  # loan term in years
extra_payment_month = 45  # The month when you want to make the extra payment
extra_payment_amount = 450000  # The amount of the extra payment
start_date = pd.to_datetime('2019-11-15')  # The start date of the mortgage

amortization_schedule_no_extra_payment = mortgage_amortization(principal, annual_interest_rate, years, 0, 0, start_date)
amortization_schedule_with_extra_payment = mortgage_amortization(principal, annual_interest_rate, years, extra_payment_month, extra_payment_amount, start_date)

pd.set_option('display.max_rows', None)
print(amortization_schedule_with_extra_payment)
#print(amortization_schedule_with_extra_payment.iloc[12*years-316])

print("\r\n --")
print("Start date: " + start_date.strftime("%d-%B-%Y"))  # prints in format "DD/MM/YYYY"
print("Principal: ${:,.2f}".format(principal))
print("Interest rate: " + str(annual_interest_rate) + "%")
print("Duration: " + str(years*12) + " month")
print("Extra payment of ${:,.2f}".format(extra_payment_amount) + " on " + (start_date + DateOffset(months=extra_payment_month)).strftime("%d-%B-%Y"))

print("\r\n --")
total_interest_no_extra_payment = amortization_schedule_no_extra_payment['Interest Payment'].sum()
total_interest_with_extra_payment = amortization_schedule_with_extra_payment['Interest Payment'].sum()
print("Total interest with no extra payment: ${:,.2f}".format(total_interest_no_extra_payment))
print("Total interest with extra payment: ${:,.2f}".format(total_interest_with_extra_payment))
print("Saved interest : ${:,.2f}".format(total_interest_no_extra_payment - total_interest_with_extra_payment))

print("\r\n --")
print("Paid off period without extra payment: " + (start_date + DateOffset(months=12 * years - 1)).strftime("%d-%B-%Y"))
print("Paid off period with extra payment: " + (start_date + DateOffset(months=amortization_schedule_with_extra_payment['Month paid'].iloc[-1] - 1)).strftime("%d-%B-%Y"))

In [ ]:
# Create a figure with 2 subplots (one above the other)
fig, (ax1, ax3) = plt.subplots(2, figsize=(10, 10))

# Plot the Principal Payment and Interest Payment on the first subplot
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Payments ($)', color=color)
ax1.plot(amortization_schedule_with_extra_payment['Date'], amortization_schedule_with_extra_payment['Principal Payment'], color='blue', label='Principal Payment')
ax1.plot(amortization_schedule_with_extra_payment['Date'], amortization_schedule_with_extra_payment['Interest Payment'], color='green', label='Interest Payment')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid()
ax1.set_ylim([0, 4000])  # Limit y-axis for ax1
ax1.set_title('Principal and Interest Payments Over Time')
ax1.legend(loc='best')  # Add legend to the first subplot

mortgage_info = "Start date: " + start_date.strftime("%d-%B-%Y") + "\nPrincipal: ${:,.2f}".format(principal) + "\nInterest rate: " + str(annual_interest_rate) + "%" + "\nDuration: " + str(years*12) + " month"
ax1.text(0.05, 0.95, mortgage_info, transform=ax1.transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'), verticalalignment='top', fontsize=10)
extra_payment_info = "Extra payment of ${:,.2f}".format(extra_payment_amount) + " on " + (start_date + DateOffset(months=extra_payment_month)).strftime("%d-%B-%Y")
ax1.text(0.55, 0.6, extra_payment_info, transform=ax1.transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'), verticalalignment='top', fontsize=10)

# Plot the Balance on the second subplot
ax3.set_xlabel('Date')
ax3.set_ylabel('Remaining Balance ($)', color='tab:red')
ax3.plot(amortization_schedule_with_extra_payment['Date'], amortization_schedule_with_extra_payment['Balance'], color='tab:red', label='Balance')
ax3.tick_params(axis='y', labelcolor='tab:red')
ax3.grid()
formatter = FuncFormatter(lambda y, _: '{:.16g}'.format(y))
ax3.yaxis.set_major_formatter(formatter)
ax3.set_title('Remaining Mortgage Balance Over Time')
ax3.legend(loc='best')  # Add legend to the second subplot

fig.tight_layout()  # Prevents overlapping of plots
plt.show()